# Convolutional Neural Network Grade Classifier

In [25]:
import json
import pandas as pd
import numpy as np
import re
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import datetime
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [2]:
# Read data from file:
with open('problems.json', 'r') as fp:
    problems_dict = json.load(fp)

## Data Preprocessing

In [3]:
# Process data
df = pd.DataFrame.from_dict(problems_dict, orient = 'index')
df.isna().sum()

problem_name     0
info             0
url              0
moves           22
dtype: int64

In [4]:
df = df.dropna()

In [11]:
def move_coordinate(d):
    # convert a move to the coorindates of the hold on the board
    s_split = re.split('(\d+)', d['Description'], maxsplit=1)
    # extra `-1` in both for 0 indexing
    w = ord(s_split[0].upper()) - 64 - 1
    h = int(s_split[1]) - 1
    
    return (h, w, 0)


def convert_moves(moves):
    array = np.zeros((18, 11, 1))
    for move in moves:
        array[move_coordinate(move)] = 1
    return array


df['Moves_array'] = df['moves'].apply(convert_moves)
problems = list(df['Moves_array'])
len(problems)

14902

In [14]:
problems = np.moveaxis(problems, -1, 1)
problems = problems.astype(np.float32)


In [6]:
# Process labels
grades = []
for problem in df['info']:
    grades.append(problem[2])
    
grade_map = {
        '5+':0,
        '6A': 1,
        '6A+': 2,
        '6B': 3,
        '6B+': 4,
        '6C': 5,
        '6C+': 6,
        '7A': 7,
        '7A+': 8,
        '7B': 9,
        '7B+': 10,
        '7C': 11,
        '7C+': 12,
        '8A': 13,
        '8A+': 14,
        '8B': 15,
        '8B+': 16,
        '8C': 17,
        '8C+': 18
    }

grades = [grade.split()[0] for grade in grades]
grades = [grade_map[grade] for grade in grades]
len(grades)

14902

## Create Custom Dataset

In [22]:
class ProblemDataset(Dataset):
    def __init__(self, data, label, transform=None):
        self.data = data
        self.transform = transform
        self.label = label
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        image_tensor = torch.from_numpy(image)
        
        
        
        
            
        label = self.label[idx]
        
        
        target = torch.tensor(label,dtype=torch.long)
        
        return image_tensor, target

In [38]:
X_train, X_test, y_train, y_test = train_test_split(problems, grades, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


In [39]:
train = ProblemDataset(X_train, y_train)
val = ProblemDataset(X_val, y_val)
test = ProblemDataset(X_test, y_test)

In [41]:
train_queue = DataLoader(train, batch_size=128)
val_queue = DataLoader(val, batch_size= 128)
test_queue = DataLoader(test, batch_size=128)

## Define CNN Model

In [26]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()  # Compulsory operation.
        self.conv1 = nn.Conv2d(1, 32, 7, stride=1, padding=3)
        self.conv2 = nn.Conv2d(32, 64, 5, stride=1, padding=2)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 15)
        

    def forward(self, x):
        
        x = self.conv1(x)
        
        # print('after conv1, x.size:', x.size())
        x = F.relu(x)  # Functions like ReLU, MaxPooling can be used in forward method as there is no weights in them to store.
        x = F.max_pool2d(x, 2)
        # print('after pool1, x.size:', x.size())
        x = self.conv2(x)
        
        # print('after conv2, x.size:', x.size())
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        # print('after pool2, x.size:', x.size())
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        # print('after flatten, x.size:', x.size())
        
        x = self.fc1(x)
        # print('after fc1, x.size:', x.size())
        x = F.relu(x)
        x = self.dropout2(x)
        logits = self.fc2(x)
        # print('logits.size:', logits.size())
        
        
        return logits

## Training

In [37]:
learning_rate = 0.1
epochs = 40

# create a network
model = ConvNet()

# utilise GPU
if torch.cuda.is_available():
    print('using GPU')
    model = model.to('cuda')
else:
    print('using CPU')

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), learning_rate)

# define loss function
criterion = nn.CrossEntropyLoss()

# training
## set model to training mode

## start training
for ep in range(epochs):
    ep_loss = 0.0
    ep_acc = 0.0
    model.train()
    for step, (x, y) in enumerate(train_queue):
        if torch.cuda.is_available():
            x = x.to('cuda')
            y = y.to('cuda')
            
        # set gradient to zero        
        optimizer.zero_grad()
        # calculate output
        p = model(x)
        _, preds = torch.max(p, 1)
        # calculate metrics
        loss = criterion(p, y)
        acc = torch.sum(preds == y).item()/len(y)
        
        ep_acc += acc
        ep_loss += loss
        
        # back-prop
        loss.backward()
        # update parameters
        optimizer.step()
        
        if step % 100 == 0:
            print('\repoch: %d step: %d loss: %.2f acc: %.4f'
                  % (ep, step, ep_loss/(step+1), ep_acc/(step+1)), end='')
    
    model.eval()
    correct = 0
    for val_step, (x, y) in enumerate(val_queue):
        x = x.to('cuda')
        y = y.to('cuda')
        p = model(x)
        _, preds = torch.max(p, 1)
        # calculate metrics
        correct += torch.sum(preds == y).item()
        
    
    
    print('\nepoch: %d loss: %.2f acc: %.4f, val acc: %.4f'
          % (ep, ep_loss/(step+1), ep_acc/(step+1), correct/len(y_val)))
        
    

using GPU
epoch: 0 step: 200 loss: 2.30 acc: 0.2439
epoch: 0 loss: 2.29 acc: 0.2431, val acc: 0.2412
epoch: 1 step: 200 loss: 2.18 acc: 0.2603
epoch: 1 loss: 2.17 acc: 0.2602, val acc: 0.2700
epoch: 2 step: 200 loss: 2.06 acc: 0.2794
epoch: 2 loss: 2.05 acc: 0.2797, val acc: 0.2630
epoch: 3 step: 200 loss: 1.97 acc: 0.3057
epoch: 3 loss: 1.97 acc: 0.3067, val acc: 0.2821
epoch: 4 step: 200 loss: 1.91 acc: 0.3125
epoch: 4 loss: 1.92 acc: 0.3113, val acc: 0.3009
epoch: 5 step: 200 loss: 1.88 acc: 0.3312
epoch: 5 loss: 1.88 acc: 0.3275, val acc: 0.2986
epoch: 6 step: 200 loss: 1.85 acc: 0.3364
epoch: 6 loss: 1.85 acc: 0.3342, val acc: 0.3036
epoch: 7 step: 200 loss: 1.82 acc: 0.3424
epoch: 7 loss: 1.83 acc: 0.3379, val acc: 0.3113
epoch: 8 step: 200 loss: 1.79 acc: 0.3493
epoch: 8 loss: 1.80 acc: 0.3462, val acc: 0.3130
epoch: 9 step: 200 loss: 1.77 acc: 0.3504
epoch: 9 loss: 1.77 acc: 0.3493, val acc: 0.3120
epoch: 10 step: 200 loss: 1.75 acc: 0.3574
epoch: 10 loss: 1.75 acc: 0.3553, val